In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
# import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
# import os

# Wykrycie dostępności GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [14]:
train_dir = "fruits_generalization_test/test"
valid_dir = "fruits_generalization_test/val"
test_dir  = "fruits_generalization_test/test"

In [15]:
train_transforms = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),  # mean (R,G,B)
                         (0.5, 0.5, 0.5))  # std  (R,G,B)
])

# Zbiór walidacyjny / testowy zwykle bez augmentacji, tylko normalizacja
test_transforms = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

In [16]:
# Zbiór treningowy
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
# Zbiór walidacyjny
valid_dataset = datasets.ImageFolder(root=valid_dir, transform=test_transforms)
# Zbiór testowy
test_dataset  = datasets.ImageFolder(root=test_dir,  transform=test_transforms)

In [17]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=2)

# Liczba klas
num_classes = len(train_dataset.classes)
print("Liczba klas:", num_classes)
print("Klasy (index -> nazwa):", train_dataset.class_to_idx)

Liczba klas: 16
Klasy (index -> nazwa): {'Apple': 0, 'Banana 3': 1, 'Beans 1': 2, 'Blackberry': 3, 'Cabbage': 4, 'Cactus fruit green 1': 5, 'Cactus fruit red 1': 6, 'Caju seed 1': 7, 'Cherry Wax not rippen 1': 8, 'Cucumber': 9, 'Gooseberry 1': 10, 'Pear': 11, 'Pistachio 1': 12, 'Zucchini': 13, 'carrot_1': 14, 'eggplant_long_1': 15}


In [18]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 100 -> 50

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 50 -> 25

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)   # 25 -> ~12
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 12 * 12, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = SimpleCNN(num_classes).to(device)
print(model)

SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=18432, out_features=256, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=256, out_features=16, bias=True)
  )
)


In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Statystyki
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [ ]:
epochs = 10

for epoch in range(1, epochs + 1):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    valid_loss, valid_acc = evaluate(model, valid_loader, criterion, device)
    
    print(f"Epoch [{epoch}/{epochs}]"
          f" | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}"
          f" | Val Loss: {valid_loss:.4f} | Val Acc: {valid_acc:.4f}")

In [13]:
test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

Test Loss: 0.0825, Test Acc: 0.9794


In [ ]:
torch.save(model.state_dict(), "fruits_cnn.pth")

# wczytanie (w nowej sesji lub innym skrypcie)
model = SimpleCNN(num_classes)
model.load_state_dict(torch.load("fruits_cnn.pth", map_location=device))
model.to(device)
model.eval()